# notebook to prepare data

In [35]:
import os
import shutil
import pandas as pd
import numpy as np

In [36]:
root_path = '/mnt/c/Arbeid/Github_Repo/MSAD_work/'

In [37]:
def concat_results(directory):
    dfs = list()
    
    # Iterate through execution dates
    for exec_folder in os.listdir(directory):
        for file in os.listdir(os.path.join(directory, exec_folder)):
            if file == 'results.csv':
                filepath = os.path.join(directory, exec_folder, file)
                df = pd.read_csv(filepath)
                df['result_path'] = filepath
                dfs.append(df)
    
    return pd.concat([df.reset_index(drop=True) for df in dfs], ignore_index=True)

directory = '/mnt/c/Arbeid/Github_Repo/TimeEval_work/results'
all_results = concat_results(directory)

In [ ]:
import pandas as pd

# Sample DataFrame creation with intentional duplicates in the first three columns
data = {
    'A': ['foo', 'foo', 'foo', 'bar', 'bar', 'bar'],
    'B': ['one', 'one', 'two', 'three', 'three', 'two'],
    'C': ['small', 'small', 'large', 'small', 'small', 'large'],
    'Value': [1, 3, 2, 5, 4, 6]  # Column based on which we'll decide which duplicate to keep
}

df = pd.DataFrame(data)

# Specify the column to base the removal of duplicates
column_of_interest = 'Value'

# Step 1: Sort the DataFrame based on the column_of_interest in descending order.
# This will ensure that the first occurrence of each duplicate set is the one with the highest 'Value'
df_sorted = df.sort_values(by=column_of_interest, ascending=False)

# Step 2: Remove duplicates based on the first 3 columns ('A', 'B', 'C'), keep the first occurrence.
# Because we sorted in descending order by 'Value', the first occurrence will be the row with the largest 'Value'
df_unique = df_sorted.drop_duplicates(subset=['A', 'B', 'C'], keep='first')

# Reset index if desired
df_unique = df_unique.reset_index(drop=True)

df, df_unique


(     A      B      C  Value
 0  foo    one  small      1
 1  foo    one  small      3
 2  foo    two  large      2
 3  bar  three  small      5
 4  bar  three  small      4
 5  bar    two  large      6,
      A      B      C  Value
 0  bar    two  large      6
 1  bar  three  small      5
 2  foo    one  small      3
 3  foo    two  large      2)

In [38]:
obsea_results = all_results[(all_results['collection'] == 'OBSEA') | (all_results['collection'] == 'OBSEA_2')]
obsea_results = obsea_results[(obsea_results['BestF1Score'].notna()) & (obsea_results['FalseNegativeRate'].notna())]
obsea_results['collection'][obsea_results['collection'] == 'OBSEA_2'] = 'OBSEA'
obsea_results['Recommendation_ACC'] =  obsea_results.apply(lambda row: max(row['BestF1Score'] - row['FalseNegativeRate'], 0), axis=1)
obsea_results = obsea_results.sort_values(by='Recommendation_ACC', ascending=False)
obsea_results = obsea_results.drop_duplicates(subset=['algorithm', 'collection', 'dataset'], keep='first')
#obsea_results = obsea_results.groupby(['algorithm', 'collection', 'dataset'])#.max().reset_index()
#obsea_results['Recommendation_ACC'] =  obsea_results.apply(lambda row: max(row['BestF1Score'] - row['FalseNegativeRate'], 0), axis=1)
obsea_results

,algorithm,collection,dataset,algo_training_type,algo_input_dimensionality,dataset_training_type,dataset_input_dimensionality,status,hyper_params_id,repetition,...,execute_postprocess_time,ROC_AUC,RANGE_PR_AUC,BestF1Score,FalseNegativeRate,result_path,train_main_time,train_preprocess_time,train_postprocess_time,Recommendation_ACC
65073,DenoisingAutoEncoder (DAE),OBSEA,2020-01-19,semi-supervised,multivariate,multivariate,semi-supervised,Status.OK,4612c4e06ea4131330882ffd0a43444a,8,...,NaN,NaN,1.0,0.999995,0.0,/mnt/c/Arbeid/Github_Repo/TimeEval_work/result...,21.416301,NaN,NaN,0.999995
65211,DenoisingAutoEncoder (DAE),OBSEA,2020-01-22,semi-supervised,multivariate,multivariate,semi-supervised,Status.OK,4612c4e06ea4131330882ffd0a43444a,1,...,NaN,NaN,1.0,0.999995,0.0,/mnt/c/Arbeid/Github_Repo/TimeEval_work/result...,16.647491,NaN,NaN,0.999995
98556,AutoEncoder (AE),OBSEA,2020-01-13,semi-supervised,multivariate,multivariate,semi-supervised,Status.OK,dc234b17555e618106e747494bd37087,1,...,NaN,NaN,1.0,0.999995,0.0,/mnt/c/Arbeid/Github_Repo/TimeEval_work/result...,19.743958,NaN,NaN,0.999995
98573,AutoEncoder (AE),OBSEA,2020-01-14,semi-supervised,multivariate,multivariate,semi-supervised,Status.OK,b98f98446ffdbf7f8a0cb80942138c22,1,...,NaN,NaN,1.0,0.999995,0.0,/mnt/c/Arbeid/Github_Repo/TimeEval_work/result...,16.850466,NaN,NaN,0.999995
65207,DenoisingAutoEncoder (DAE),OBSEA,2020-01-21,semi-supervised,multivariate,multivariate,semi-supervised,Status.OK,c8d37c7a26a5f51260539d3c90e37648,10,...,NaN,NaN,1.0,0.999995,0.0,/mnt/c/Arbeid/Github_Repo/TimeEval_work/result...,19.345597,NaN,NaN,0.999995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99409,AutoEncoder (AE),OBSEA,2020-03-23,semi-supervised,multivariate,multivariate,semi-supervised,Status.OK,dfdc9a9fdc00b09d9d188aaabfa8f91c,2,...,NaN,NaN,NaN,0.000000,0.0,/mnt/c/Arbeid/Github_Repo/TimeEval_work/result...,16.490262,NaN,NaN,0.000000
99455,AutoEncoder (AE),OBSEA,2020-03-26,semi-supervised,multivariate,multivariate,semi-supervised,Status.OK,b98f98446ffdbf7f8a0cb80942138c22,1,...,NaN,NaN,NaN,0.000000,0.0,/mnt/c/Arbeid/Github_Repo/TimeEval_work/result...,18.400768,NaN,NaN,0.000000
99359,AutoEncoder (AE),OBSEA,2020-03-20,semi-supervised,multivariate,multivariate,semi-supervised,Status.OK,b98f98446ffdbf7f8a0cb80942138c22,3,...,NaN,NaN,NaN,0.000000,0.0,/mnt/c/Arbeid/Github_Repo/TimeEval_work/result...,16.534390,NaN,NaN,0.000000
99371,AutoEncoder (AE),OBSEA,2020-03-21,semi-supervised,multivariate,multivariate,semi-supervised,Status.OK,b98f98446ffdbf7f8a0cb80942138c22,2,...,NaN,NaN,NaN,0.000000,0.0,/mnt/c/Arbeid/Github_Repo/TimeEval_work/result...,17.496970,NaN,NaN,0.000000


In [5]:
#obsea_results = obsea_results[:500]
#obsea_results

In [6]:
obsea_results['algorithm'].unique()

array(['AutoEncoder (AE)', 'CBLOF', 'COPOD', 'DeepAnT',
       'DenoisingAutoEncoder (DAE)', 'EncDec-AD', 'HBOS', 'Hybrid KNN',
       'LOF', 'PCC', 'Random Black Forest (RR)', 'RobustPCA', 'Torsk'],
      dtype=object)

In [7]:
# copy data
dest_path = root_path + 'data/OBSEA/data'
src_path = '/mnt/c/Arbeid/Github_Repo/TimeEval_work/work_data/processed/multivariate/OBSEA'
for data_file in os.listdir(src_path):
    if 'test' in data_file:
        df = pd.read_csv(src_path + '/' + data_file)
        df.drop(df.columns[0], axis=1, inplace=True)
        df.to_csv(dest_path + '/' + data_file.split('.')[0] + '.out', index=False, header=False)

In [46]:
# copy scores
dest_path = root_path + 'data/OBSEA/scores/OBSEA'
for row_idx in range(len(obsea_results)):
    result_path = '/'.join(obsea_results.iloc[row_idx, obsea_results.columns.get_loc('result_path')].split('/')[:-1])
    algorithm = obsea_results.iloc[row_idx, obsea_results.columns.get_loc('algorithm')]
    hyper_params_id = obsea_results.iloc[row_idx, obsea_results.columns.get_loc('hyper_params_id')]
    collection = obsea_results.iloc[row_idx, obsea_results.columns.get_loc('collection')]
    dataset = obsea_results.iloc[row_idx, obsea_results.columns.get_loc('dataset')]
    repetition = str(obsea_results.iloc[row_idx, obsea_results.columns.get_loc('repetition')])
    scores_file_name = 'docker-algorithm-scores.csv'
    src_path = result_path + '/' + algorithm + '/' + hyper_params_id + '/' + collection + '/' + dataset + '/' + repetition + '/' + scores_file_name
    scores_dir_alg = dest_path + '/' + algorithm + '/score'
    if '(AE)' in scores_dir_alg:
        scores_dir_alg = scores_dir_alg.replace(' ', '_')
    os.makedirs(scores_dir_alg, exist_ok=True)
    dest_score_path = scores_dir_alg + '/' + dataset.split('_')[0] + '.out'
    if os.path.exists(src_path):
        shutil.copy(src_path, dest_score_path)
    else:
        src_path = src_path.replace('OBSEA', 'OBSEA_2')
        shutil.copy(src_path, dest_score_path)

In [25]:
# copy metrics
path_to_metrics = root_path + 'data/OBSEA/metrics'
metrics = ['ROC_AUC', 'RANGE_PR_AUC', 'BestF1Score', 'FalseNegativeRate', 'Recommendation_ACC']
exec_algorithms = obsea_results['algorithm'].unique()
for exec_algorithm in exec_algorithms:
    for metric in metrics:
        df = obsea_results[(obsea_results['algorithm'] == exec_algorithm)]
        df = df[['dataset', metric]]
        df['dataset'] = df['dataset'].str.strip().str.replace('_unsupervised', '.out')
        df['dataset'] = df['dataset'].apply(lambda x: x + '.out' if not x.endswith('.out') else x)
        df['dataset'] = df['dataset'].apply(lambda x: 'OBSEA/' + x)
        df.set_index('dataset', inplace=True)
        df.index.name = ''
        df = df.groupby(level=0).min()
        df.rename(columns={metric: exec_algorithm}, inplace=True)
        df.dropna(inplace=True)
        metrics_dir_alg = path_to_metrics + '/' + exec_algorithm
        os.makedirs(metrics_dir_alg, exist_ok=True)
        #print(df)
        df.to_csv(metrics_dir_alg + '/' + metric + '.csv')